In [18]:
import prep
import prsa
import os
import numpy as np
import biosppy

In [19]:
FOLDER_NAME = 'biosppy_results' # folder to save results in .csv format

In [20]:
NORMO_DIR = './data_2/normokapnia'
HYPER_DIR = './data_2/hiperkapnia'

ABP_COLUMN = 'abp_finger_mm_hg_[abp_finger_mm_Hg_]'
ALTERNATIVE_ABP_COLUMN = 'abp_finger[abp_finger]'

In [21]:
normo_dc_data = {}
normo_ac_data = {}
hyper_dc_data = {}
hyper_ac_data = {}

normo_dc_w = {}
normo_ac_w = {}
hyper_dc_w = {}
hyper_ac_w = {}

In [22]:
for filename in os.listdir(NORMO_DIR):
    if os.path.isfile(os.path.join(NORMO_DIR, filename)):
        file_path = os.path.join(NORMO_DIR, filename)
        try:
            df = prep.read_data(file_path, ABP_COLUMN, sep=';')
        except KeyError:
            df = prep.read_data(file_path, ALTERNATIVE_ABP_COLUMN, sep=';')

        try:
            df.interpolate(method='linear', inplace=True)
            filtered_abp = biosppy.signals.abp.abp(signal=df['Values'], sampling_rate=200, show=False)[1]
            peaks_indexs = biosppy.signals.abp.find_onsets_zong2003(signal=filtered_abp, sampling_rate=200, d1_th = 30, d2_th = 0.1)[0]

            # makes array of time at wich peaks occured
            peaks_time = df["TimeSteps"].iloc[peaks_indexs].to_numpy() * 0.005
            # calculates intervals between peaks
            rr_intervals = np.array(np.diff(peaks_time))
            prsa_dc, dc_w = prsa.calculate(rr_intervals, 3, "DC", 0.2)
            prsa_ac, ac_w = prsa.calculate(rr_intervals, 3, "AC", 0.2)
            dc= prsa.capacity(prsa_dc)
            ac = prsa.capacity(prsa_ac)
        except:
            print(filename)
            continue
        else:
            filename = filename[-7:-5]
            normo_dc_data[filename] = dc
            normo_ac_data[filename] = ac
            normo_dc_w[filename] = dc_w
            normo_ac_w[filename] = ac_w

e[3]_[n]_[23].csv
e[3]_[n]_[25].csv


c:\Users\user\PycharmProjects\Brainlab\venv\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\user\PycharmProjects\Brainlab\venv\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


e[3]_[n]_[29].csv
e[3]_[n]_[31].csv
e[3]_[n]_[35].csv
e[3]_[n]_[41].csv
e[3]_[n]_[48].csv


In [23]:
for filename in os.listdir(HYPER_DIR):
    if os.path.isfile(os.path.join(HYPER_DIR, filename)):
        file_path = os.path.join(HYPER_DIR, filename)
        try:
            df = prep.read_data(file_path, ABP_COLUMN, sep=';')
        except KeyError:
            df = prep.read_data(file_path, ALTERNATIVE_ABP_COLUMN, sep=';')

        try:
            df.interpolate(method='linear', inplace=True)
            filtered_abp = biosppy.signals.abp.abp(signal=df['Values'], sampling_rate=200, show=False)[1]
            peaks_indexs = biosppy.signals.abp.find_onsets_zong2003(signal=filtered_abp, sampling_rate=200, d1_th = 30, d2_th = 0.1)[0]

            # makes array of time at wich peaks occured
            peaks_time = df["TimeSteps"].iloc[peaks_indexs].to_numpy() * 0.005
            # calculates intervals between peaks
            rr_intervals = np.array(np.diff(peaks_time))
            prsa_dc, dc_w = prsa.calculate(rr_intervals, 3, "DC", 0.2)
            prsa_ac, ac_w = prsa.calculate(rr_intervals, 3, "AC", 0.2)
            dc = prsa.capacity(prsa_dc)
            ac = prsa.capacity(prsa_ac)
        except:
            print(filename)
            continue
        else:
            filename = filename[-7:-5]
            hyper_dc_data[filename] = dc
            hyper_ac_data[filename] = ac
            hyper_dc_w[filename] = dc_w
            hyper_ac_w[filename] = ac_w

e[3]_[h]_[04].csv
e[3]_[h]_[09].csv
e[3]_[h]_[14].csv
e[3]_[h]_[25].csv
e[3]_[h]_[35].csv
e[3]_[h]_[41].csv
e[3]_[h]_[47].csv
e[3]_[h]_[48].csv


In [58]:
def save_to_csv(file_tittle: str, folder_name: str, **columns: dict[str: float]):
    """
    Saves results to csv file
    :param file_tittle: name of the file
    :param folder_name: name of the folder
    :param columns: column names and their values for given file
    """

    # Get the column names and values
    columns_names = list(columns.keys())
    columns_values = list(columns.values())

    # Get the filenames
    filenames = set()
    for col in columns_values:
        filenames.update(list(col.keys()))

    filenames = list(filenames)
    filenames.sort(key=lambda x: int(x))
    # Create the directory if it doesn't exist
    os.makedirs(folder_name, exist_ok=True)

    with open(f'{folder_name}/{file_tittle}.csv', 'w') as f:
        f.write(f'file number;{";".join(columns_names)}\n')
        for filename in filenames:
            row_values = [row.get(filename, -np.inf) for row in columns_values]
            if -np.inf in row_values:
                continue
            f.write(f'{filename};{";".join([str(val) for val in row_values])}\n')


In [59]:
save_to_csv(
    file_tittle = 'AC_DC_Values',
    folder_name = FOLDER_NAME,
    AC_normo = normo_ac_data, 
    AC_hyper = hyper_ac_data,
    DC_normo = normo_dc_data, 
    DC_hyper = hyper_dc_data,
    )

In [60]:
save_to_csv(
    file_tittle = 'AC_DC_Windows',
    folder_name = FOLDER_NAME,
    AC_normo_W = normo_ac_w,
    AC_hyper_W = hyper_ac_w,
    DC_normo_W = normo_dc_w,
    DC_hyper_W = hyper_dc_w
    )